In [1]:
!rm info/*

rm: cannot remove ‘info/*’: No such file or directory


In [2]:
# youtubednn嵌入部分模型训练逻辑
# 基础依赖
import argparse
import pickle
import boto3
import os
import numpy as np
import itertools
import tarfile
from tqdm import tqdm
import pandas as pd
# 模型相关
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, save_model, load_model
import faiss
from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss
from deepmatch.utils import recall_N

In [3]:
########################################
# 从s3同步数据
########################################
def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))
        
def write_to_s3(filename, bucket, key):
    with open(filename, 'rb') as f:  # Read in binary mode
        return s3client.upload_fileobj(f, bucket, key)

default_bucket = 'sagemaker-us-east-1-002224604296'
default_mk_region = '1'
level_1 = 'recommender-system-film-mk'

parser = argparse.ArgumentParser()
parser.add_argument('--bucket', type=str, default=default_bucket)
parser.add_argument('--mk-region', type=str, default=default_mk_region)

args, _ = parser.parse_known_args()
bucket = args.bucket
mk_region = args.mk_region

prefix = f"{level_1}/{mk_region}"

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

s3client = boto3.client('s3')
local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)
# 行为数据加载
file_name_list = ['action.csv']
s3_folder = '{}/system/user-data/clean/latest'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
# encode映射数据
file_name_list = ['raw_embed_item_mapping.pickle', 'raw_embed_user_mapping.pickle']
s3_folder = '{}/feature/action/'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
# 加载pickle文件
file_to_load = open("info/raw_embed_item_mapping.pickle", "rb")
raw_embed_item_mapping = pickle.load(file_to_load)
file_to_load = open("info/raw_embed_user_mapping.pickle", "rb")
raw_embed_user_mapping = pickle.load(file_to_load)

bucket=sagemaker-us-east-1-002224604296
prefix='recommender-system-film-mk/1'
file preparation: download src key recommender-system-film-mk/1/system/user-data/clean/latest/action.csv to dst key info/action.csv
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_item_mapping.pickle to dst key info/raw_embed_item_mapping.pickle
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_user_mapping.pickle to dst key info/raw_embed_user_mapping.pickle
load 207341 action data


In [4]:
data_mk = pd.read_csv('info/action.csv',sep='\t')
data_mk.head()
data_mk[data_mk['userid']=='h50e7e4eb0891']

,label,userid,programId,programType,action,timeStamp,title,genres
6714,1,h50e7e4eb0891,1681277,1,2,1613782683,becoming,documentary
7241,1,h50e7e4eb0891,1681277,1,1,1613782674,becoming,documentary
17784,0,h50e7e4eb0891,1681278,1,1,1613782664,father soldier son,documentary|war
27107,0,h50e7e4eb0891,1647202,1,1,1613782664,game of thrones s08e06,action|adventure|drama
83744,0,h50e7e4eb0891,1676922,1,1,1613782664,les plus belles annees dune vie,drama
109462,0,h50e7e4eb0891,1692202,1,1,1613782676,15 august,drama


In [5]:
data_tt = data_mk[['userid','programId','title','genres','timeStamp','label']].rename(columns={'userid':'user_id','programId':'movie_id','timeStamp':'timestamp','label':'rating'})

In [6]:
data_tt = data_tt.astype('string')

In [7]:
data_tt_click = data_tt[data_tt['rating']=='1']
data_tt_exp = data_tt[data_tt['rating']=='0']
print('click len {} and exp len {}, click rate {}, {} movies'.format(len(data_tt_click),len(data_tt_exp),float(len(data_tt_click)/len(data_tt)),
len(data_tt['movie_id'].unique())))

features = ['user_id', 'movie_id']
map_dicts = [raw_embed_user_mapping, raw_embed_item_mapping]
feature_max_idx = {}
for feature, map_dict in zip(features, map_dicts):
#     lbe = LabelEncoder()
#     data_tt[feature] = lbe.fit_transform(data_tt[feature]) + 1
    data_tt[feature] = data_tt[feature].apply(lambda x: map_dict[x])
    feature_max_idx[feature] = data_tt[feature].max() + 1

click len 10851 and exp len 196490, click rate 0.05233407767879966, 1679 movies


In [8]:
# # constructu mapping dictionary
# raw_user_id_list = list(map(str,data_tt_raw['user_id'].values))
# code_user_id_list = list(map(str,data_tt['user_id'].values))
# raw_embed_user_id_dict = dict(zip(raw_user_id_list, code_user_id_list))
# embed_raw_user_id_dict = dict(zip(code_user_id_list, raw_user_id_list))

# raw_item_id_list = list(map(str,data_tt_raw['movie_id'].values))
# code_item_id_list = list(map(str,data_tt['movie_id'].values))
# raw_embed_item_id_dict = dict(zip(raw_item_id_list, code_item_id_list))
# embed_raw_item_id_dict = dict(zip(code_item_id_list, raw_item_id_list))

In [9]:
item_profile = list(raw_embed_item_mapping.keys())
item_profile_encode = []
for ele in item_profile:
    item_profile_encode.append(raw_embed_item_mapping[ele])

In [10]:
count_thred = 2
data_tt_filter = data_tt[data_tt['rating']=='1'].groupby(['movie_id']).filter(lambda x: len(x) > count_thred)
pass_item_pddf = data_tt_filter.drop_duplicates('movie_id')[['movie_id','rating']]

data_tt_click = data_tt[data_tt['rating']=='1']
keys = list(pass_item_pddf.columns.values)
i1 = data_tt_click.set_index(keys).index
i2 = pass_item_pddf.set_index(keys).index
data_tt_click = data_tt_click[i1.isin(i2)]

In [11]:
#data = pd.read_csvdata = pd.read_csv("./movielens_sample.txt")
sparse_features = ["movie_id", "user_id"]
SEQ_LEN = 50
negsample = 5

# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`
user_profile = data_tt[["user_id"]].drop_duplicates('user_id')

# item_profile = data_tt.drop_duplicates('movie_id')
# item_profile = data[["movie_id"]].drop_duplicates('movie_id')

user_profile.set_index("user_id", inplace=True)

user_item_list = data_tt.groupby("user_id")['movie_id'].apply(list)

# train_set, test_set = gen_data_set(data_tt_click, negsample, data_tt['movie_id'].unique())
train_set, test_set = gen_data_set(data_tt_click, negsample, np.array(item_profile_encode))

train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

# 2.count #unique features for each sparse field and generate feature config for sequence feature

embedding_dim = 32

user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]

# 3.Define Model and train

K.set_learning_phase(True)

import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()

model = YoutubeDNN(user_feature_columns, item_feature_columns, num_sampled=100, user_dnn_hidden_units=(128,64, embedding_dim))
# model = MIND(user_feature_columns,item_feature_columns,dynamic_k=False,p=1,k_max=2,num_sampled=100,user_dnn_hidden_units=(128,64, embedding_dim))

model.compile(optimizer="adam", loss=sampledsoftmaxloss)  # "binary_crossentropy")

history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=300, verbose=1, validation_split=0.0, )

# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
# all_item_model_input = {"movie_id": item_profile['movie_id'].values,}
all_item_model_input = {"movie_id": np.array(item_profile_encode),}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

# print(user_embs.shape)
# print(item_embs.shape)
# print(user_embs)

100%|██████████| 1779/1779 [00:15<00:00, 117.74it/s]


5 6
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 41142 samples
Epoch 1/300
41142/41142 [==============================] - 2s 44us/sample - loss: 3.2870
Epoch 2/300
41142/41142 [==============================] - 1s 16us/sample - loss: 2.9953
Epoch 3/300
41142/41142 [==============================] - 1s 16us/sample - loss: 2.9368
Epoch 4/300
41142/41142 [==============================] - 1s 16us/sample - loss: 2.9541
Epoch 5/300
41142/41142 [==============================] - 1s 16us/sample - loss: 2.9003
Epoch 6/300
41142/41142 [==============================] - 1s 16us/sample - loss: 2.9098
Epoch 7/300
41142/41142 [==============================] - 1s 16us/sample - loss: 2.9374


Epoch 76/300
41142/41142 [==============================] - 1s 16us/sample - loss: 1.4208
Epoch 77/300
41142/41142 [==============================] - 1s 17us/sample - loss: 1.4238
Epoch 78/300
41142/41142 [==============================] - 1s 17us/sample - loss: 1.3692
Epoch 79/300
41142/41142 [==============================] - 1s 17us/sample - loss: 1.3133
Epoch 80/300
41142/41142 [==============================] - 1s 17us/sample - loss: 1.3479
Epoch 81/300
41142/41142 [==============================] - 1s 17us/sample - loss: 1.2719
Epoch 82/300
41142/41142 [==============================] - 1s 16us/sample - loss: 1.2256
Epoch 83/300
41142/41142 [==============================] - 1s 17us/sample - loss: 1.3183
Epoch 84/300
41142/41142 [==============================] - 1s 17us/sample - loss: 1.3043
Epoch 85/300
41142/41142 [==============================] - 1s 17us/sample - loss: 1.2570
Epoch 86/300
41142/41142 [==============================] - 1s 17us/sample - loss: 1.2569
Epoch 87/3

41142/41142 [==============================] - 1s 15us/sample - loss: 0.2836
Epoch 257/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.2830
Epoch 258/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.2458
Epoch 259/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.2585
Epoch 260/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.2324
Epoch 261/300
41142/41142 [==============================] - 1s 16us/sample - loss: 0.2269
Epoch 262/300
41142/41142 [==============================] - 1s 16us/sample - loss: 0.2827
Epoch 263/300
41142/41142 [==============================] - 1s 16us/sample - loss: 0.2433
Epoch 264/300
41142/41142 [==============================] - 1s 16us/sample - loss: 0.2554
Epoch 265/300
41142/41142 [==============================] - 1s 16us/sample - loss: 0.2365
Epoch 266/300
41142/41142 [==============================] - 1s 16us/sample - loss: 0.2448
Epoch 267/300

In [12]:
index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0
test_cnt = 0

test_true_label = {line[0]:[line[2]] for line in test_set}

for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
#     try:
        pred = [item_profile_encode[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
#         print(test_true_label[uid])
#         print(pred)
#         break
        if test_true_label[uid][0] in pred:
#             if test_cnt < 10:
#                 print("input {} pred {} and label {}".format(I[i], pred, test_true_label[uid]))
#                 test_cnt = test_cnt + 1
            hit += 1
#         if recall_score > 0:
#             print("input {} pred {} and label {}".format(I[i], pred, test_true_label[uid]))
#             test_cnt = test_cnt + 1
#     except:
#         print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))

1713it [00:00, 49634.16it/s]


recall 0.6398131932282545
hit rate 0.6398131932282545


In [13]:
# 存储/更新用户嵌入模型
file_name = 'info/user_embeddings.h5'
save_model(user_embedding_model, file_name)
write_to_s3(file_name, bucket, "{}/model/recall/youtubednn/{}".format(prefix,file_name.split('/')[-1]))

# 存储/更新faiss index
file_name = 'info/ub_item_vector.index'
faiss.write_index(index, file_name)
write_to_s3(file_name, bucket, "{}/feature/action/{}".format(prefix,file_name.split('/')[-1]))

# # 存储/更新映射pickle文件
# file_name = 'info/raw_embed_user_mapping.pickle'
# output_file = open(file_name, 'wb')
# pickle.dump(raw_embed_user_id_dict, output_file)
# output_file.close()
# write_to_s3(file_name, bucket, "{}/feature/action/{}".format(prefix,file_name.split('/')[-1]))

# file_name = 'info/embed_raw_user_mapping.pickle'
# output_file = open(file_name, 'wb')
# pickle.dump(embed_raw_user_id_dict, output_file)
# output_file.close()
# write_to_s3(file_name, bucket, "{}/feature/action/{}".format(prefix,file_name.split('/')[-1]))

# file_name = 'info/raw_embed_item_mapping.pickle'
# output_file = open(file_name, 'wb')
# pickle.dump(raw_embed_item_id_dict, output_file)
# output_file.close()
# write_to_s3(file_name, bucket, "{}/feature/action/{}".format(prefix,file_name.split('/')[-1]))

# file_name = 'info/embed_raw_item_mapping.pickle'
# output_file = open(file_name, 'wb')
# pickle.dump(embed_raw_item_id_dict, output_file)
# output_file.close()
# write_to_s3(file_name, bucket, "{}/feature/action/{}".format(prefix,file_name.split('/')[-1]))

# 存储/更新物品embedding npy
file_name = 'info/ub_item_embeddings.npy'
with open(file_name,'wb') as f:
    np.save(f, item_embs)
write_to_s3(file_name, bucket, "{}/feature/action/{}".format(prefix,file_name.split('/')[-1]))

In [1]:
!rm -r info/*
!python model-update-ub.py


bucket=sagemaker-us-east-1-002224604296
prefix='recommender-system-film-mk/1'
file preparation: download src key recommender-system-film-mk/1/system/user-data/clean/latest/action.csv to dst key info/action.csv
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_item_mapping.pickle to dst key info/raw_embed_item_mapping.pickle
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_user_mapping.pickle to dst key info/raw_embed_user_mapping.pickle
click len 10851 and exp len 196490, click rate 0.05233407767879966, 1679 movies
100%|██████████████████████████████████████| 1779/1779 [00:14<00:00, 122.25it/s]
5 6
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in f

Epoch 66/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.7840
Epoch 67/300
41142/41142 [==============================] - 1s 16us/sample - loss: 1.7198
Epoch 68/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.7284
Epoch 69/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.7183
Epoch 70/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.6704
Epoch 71/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.6898
Epoch 72/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.6016
Epoch 73/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.6147
Epoch 74/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.5576
Epoch 75/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.5489
Epoch 76/300
41142/41142 [==============================] - 1s 15us/sample - loss: 1.5311
Epoch 77/3

41142/41142 [==============================] - 1s 15us/sample - loss: 0.5041
Epoch 157/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.4917
Epoch 158/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.5118
Epoch 159/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.5618
Epoch 160/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.5311
Epoch 161/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.5358
Epoch 162/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.5390
Epoch 163/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.5147
Epoch 164/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.4922
Epoch 165/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.5283
Epoch 166/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.4660
Epoch 167/300

41142/41142 [==============================] - 1s 15us/sample - loss: 0.3394
Epoch 247/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.3021
Epoch 248/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.3117
Epoch 249/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.3136
Epoch 250/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.2908
Epoch 251/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.3047
Epoch 252/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.2748
Epoch 253/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.2808
Epoch 254/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.2654
Epoch 255/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.2677
Epoch 256/300
41142/41142 [==============================] - 1s 15us/sample - loss: 0.3023
Epoch 257/300